In [1]:
import requests
from io import StringIO
import pandas as pd
import numpy as np

In [2]:
import datetime
DAY = datetime.timedelta(1)

In [3]:
def Day_Stock(date_str):
    r = requests.post('https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + date_str + '&type=ALL')
    df = pd.read_csv(StringIO(r.text.replace("=", "")), header=["證券代號" in l for l in r.text.split("\n")].index(True)-1)
    df = df.apply(lambda s: s.astype(str).str.replace(",", "").replace("+", "1").replace("-", "-1"))
    df = df.set_index("證券代號")
    df.drop("Unnamed: 16", axis=1, inplace=True)
    return df

In [4]:
dfs = {}
for i in range(7):
    target_date = datetime.datetime.now() - i * DAY
    weekday = target_date.strftime('%w')
    target_date_str = target_date.strftime('%Y%m%d')
    if weekday not in ['0', '6']:
        dfs[target_date_str] = Day_Stock(target_date_str)

In [5]:
target = {}
for date in dfs:
    target[date] = dfs[date].loc['0050']

In [6]:
target_df = pd.DataFrame(target).transpose()

In [7]:
target_df.set_index(pd.to_datetime(target_df.index))

,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
2020-06-09,元大台灣50,11356579,4738,1016232445,89.60,89.80,88.95,89.60,1,0.1,89.60,235,89.65,44,0.00
2020-06-08,元大台灣50,15549542,7617,1388746538,88.90,89.80,88.80,89.50,1,1.15,89.45,19,89.50,230,0.00
2020-06-05,元大台灣50,9916582,5501,873460722,87.70,88.35,87.70,88.35,1,0.75,88.30,624,88.35,191,0.00
2020-06-04,元大台灣50,15036553,6182,1315498861,87.50,87.75,87.15,87.60,1,0.8,87.55,720,87.60,45,0.00
2020-06-03,元大台灣50,18117244,7251,1567235805,86.15,86.90,86.10,86.80,1,1.5,86.75,535,86.80,20,0.00


In [ ]:


datestr = datetime.datetime.now().strftime('%Y%m%d')

# 下載股價
r = requests.post('https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + datestr + '&type=ALL')



In [ ]:
# 整理資料，變成表格
df = pd.read_csv(StringIO(r.text.replace("=", "")), 
            header=["證券代號" in l for l in r.text.split("\n")].index(True)-1)

# 整理一些字串：
df = df.apply(lambda s: s.astype(str).str.replace(",", "").replace("+", "1").replace("-", "-1"))

# 顯示出來


In [ ]:
df = df.set_index("證券代號")
df.drop("Unnamed: 16", axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df = df.astype(
    {
        '成交股數': np.float,
        '成交筆數': np.float,
        '成交金額': np.float,
        '開盤價': np.float,
        '最高價': np.float,
        '最低價': np.float,
        '收盤價': np.float,
        '漲跌(+/-)': np.float,
        '漲跌價差': np.float,
        '最後揭示買價': np.float,
        '最後揭示買量': np.float,
        '最後揭示賣價': np.float,
        '最後揭示賣量': np.float,
        '本益比': np.float
    },
    errors='ignore')

In [ ]:
df[pd.to_numeric(df['本益比'], errors='coerce')<15]